# Example - Estimate Excess Surface Density with stacking strategy

we import some clmm modules

In [17]:
import sys
import os
import fnmatch
import pickle
os.environ['CLMM_MODELING_BACKEND'] = 'nc' # here you may choose ccl or nc (NumCosmo)
sys.path.append('/pbs/throng/lsst/users/cpayerne/CLMM/examples/support')
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from numpy import random

clmm.__version__

'0.3.0'

we import `clmm` module's core.

In [18]:
import clmm.polaraveraging as pa
import clmm.galaxycluster as gc
import clmm.modeling as modeling
from clmm import Cosmology 

we import module for mock data

In [19]:
import mock_data as mock

basic modules

In [20]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from scipy.optimize import curve_fit

we import `DC2Analysis` modules

In [21]:
import sys
sys.path.append('/pbs/throng/lsst/users/cpayerne/GitForThesis/DC2Analysis')
import stacking as st
import modeling as model
import utils as ut

In [22]:
def load(filename, **kwargs):
    
    """Loads GalaxyCluster object to filename using Pickle"""
    
    with open(filename, 'rb') as fin:
        
        return pickle.load(fin, **kwargs)

for reproductibility :

In [23]:
np.random.seed(11)

we define cosmology into `Cosmology` object

In [24]:
mock_cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

we define the `Modeling` object `moo` to for the galaxy cluster.

In [25]:
moo = clmm.Modeling(massdef = 'mean', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(mock_cosmo)

we define the `Stacking` object `stack` by definning a given range of radial axis, the number of bins for the binned stacked shear profile.

In [26]:
stack = st.Stacking(r_low = 0.5,  r_up = 6, n_bins = 10, cosmo = mock_cosmo)

we now define the type of desired profile : tangential reduced shear of Excess surface density for the stacking procedure by modifying the atribute. We use the method `Type` from the class to select DeltaSigma of reduced tangential shear.

In [27]:
deltasigma = True

In [28]:
stack._select_type(is_deltasigma = deltasigma)

We now define some redshift interval and input mass interval to select GalaxyCluster object in preselected galaxy catalogs from `cosmoDC2`

In [29]:
z_bin = [0.4,0.5]
m_bin = [10**14, 1.2*10**14]
m_bin = np.array(m_bin)
n_catalogs = 10
ngals = 2000

we define a set of selected cluster with given true masses ans true redshifts

In [30]:
cluster_m = (m_bin[1] - m_bin[0])*np.random.random(n_catalogs) + m_bin[0] #in M_sun
cluster_z = (z_bin[1] - z_bin[0])*np.random.random(n_catalogs) + z_bin[0]

We add each galaxy catalogs that correponds to binning to the `stack` object

In [ ]:
where_mock = '/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/GalaxyClusterCatalogs/cosmoDC2'

for file in os.listdir(where_mock): 
            
    if fnmatch.fnmatch(file, 'cluster*'):
    
        os.chdir(where_mock)
        
        cl = load(file)
        
        print(cl.galcat)

       # """Add tangential and cross components"""

        # the critical surface density is recquired then set is_deltasigma = True whatever you estimate reduced shear or deltasigma

        #cl.compute_tangential_and_cross_components(geometry="flat", is_deltasigma = True, cosmo = stack.cosmo)

        """Create profile"""

        #bin_edges = pa.make_bins( stack.r_low , stack.r_up , stack.n_bins , method='evenlog10width')

        #profile = cl.make_binned_profile("radians", "Mpc", bins=bin_edges,cosmo=stack.cosmo,include_empty_bins= True,gal_ids_in_bins=True)

        """Estimate individual shear profile"""

        #stack._estimate_individual_lensing_signal(cl, profile)

        #x_list.append(profile['radius'])

        """Add catalog of background galaxies to stack"""

        #stack._add_cluster_redshift(cl)

        #stack._add_background_galaxies(cl, profile)

        ra                 dec         ...         kappa          host_halo_id
------------------ ------------------- ... ---------------------- ------------
62.317187838287424  -29.55546652884356 ...   0.003286028956128728 527700056347
 62.34111165221257 -29.514834859667786 ...  -0.005081748508516837 527700056347
 62.03062493403989 -29.758603178018042 ...   0.029238711373703485 527700056347
62.253700539953314 -29.524060527439648 ...  -0.005005308144826692 527700056347
62.116019088685874   -29.6189531771933 ...  -0.006877957216113506 527700056347
 62.11598915937784 -29.767545686809818 ... -0.0025336123593140414 527700056347
62.151820978967145 -29.726990144275174 ...    -0.0051381355796003 527700056347
 62.14159529947251 -29.646217326497432 ...   0.011656930830790335 527700056347
  62.2405704014869 -29.577032210630023 ...  -0.007427456914617103 527700056347
61.979192261427045 -29.629071539544004 ...    0.02162046225508707 527700056347
               ...                 ... ...          